<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
import subprocess

# Define the URL
url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"

# Define the command
command = ["wget", url]

# Run the command
subprocess.run(command)

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [93]:
# Step 1: Import libraries and modules
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import Adam

In [88]:
# Step 2: Set parameters
params = {
    'image_size': 224,
    'batch_size': 100,
    'num_epochs': 1,
    'class_mode': 'categorical',
    'num_classes': 2,
    'shuffle': True}

In [82]:
# Step 3: Create ImageDataGenerators for training and validation sets

def batch_generator(path, preprocess_input, params):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)
    
    generator = data_generator.flow_from_directory(
        path,
        target_size=(params['image_size'], 
                     params['image_size']),
        batch_size=params['batch_size'],
        class_mode=params['class_mode'],
        shuffle=params['shuffle'])
    
    return generator

train_generator = batch_generator(
    'concrete_data_week4/train',
    preprocess_input,
    params)

validation_generator = batch_generator(
    'concrete_data_week4/valid',
    preprocess_input,
    params)

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [83]:
# Step 4: Create a sequential model
model = Sequential()

# Add VGG16 model to the sequential model
model.add(VGG16(weights = 'imagenet', include_top = False, pooling = 'avg'))
model.layers[0].trainable = False

# Add dense layer on top of VGG16
model.add(Dense(params['num_classes'], activation = 'softmax'))

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_24 (Dense)            (None, 256)               131328    
                                                                 
 dense_25 (Dense)            (None, 2)                 514       
                                                                 
Total params: 14,846,530
Trainable params: 131,842
Non-trainable params: 14,714,688
_________________________________________________________________


In [84]:
# Step 5: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [89]:
# Step 6: Fit the model on augmented data using ImageDataGenerators
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    batch_size=100,
    epochs=params['num_epochs'],
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
    workers=10
)

301/301 [==============================] - 2095s 7s/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 0.0106 - val_accuracy: 0.9972


In [91]:
# Step 7: Save the model
model.save('model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
model = load_model(filepath='model.h5')

In [101]:
params['shuffle'] = False

test_vgg16_generator = batch_generator(
    'concrete_data_week4/test',
    preprocess_input, params)

Found 500 images belonging to 2 classes.


In [103]:
result_vgg16 = model.evaluate_generator(
    test_vgg16_generator,
    verbose=1)

C:\Users\PADRÂO TI\AppData\Local\Temp\ipykernel_17696\2039740823.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  result_vgg16 = model.evaluate_generator(


5/5 [==============================] - 26s 5s/step - loss: 0.0060 - accuracy: 0.9980


In [104]:
print(f"Accuracy: {result_vgg16[1]}")
print(f"Loss: {result_vgg16[0]}")

Accuracy: 0.9980000257492065
Loss: 0.00602583447471261


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [105]:
def get_predictions(predictions):
    for pred in predictions:
        position = np.argmax(pred)
        if position == 1:
            print("Positive")
        if position == 0:
            print("Negative")

In [106]:
vgg16_prediction = model.predict_generator(
    test_vgg16_generator,
    verbose=1)

C:\Users\PADRÂO TI\AppData\Local\Temp\ipykernel_17696\900310396.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg16_prediction = model.predict_generator(


5/5 [==============================] - 26s 5s/step


In [107]:
get_predictions(vgg16_prediction[:5])

Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).